In [14]:
from IPython.display import Image
from IPython.core.display import HTML 
A1=Image(url= "https://numpy.org/_static/numpy_logo.png")
A2=Image(url="https://www.scipy.org/_static/logo.png")
A3=Image(url="https://lh3.googleusercontent.com/KMyA2BL8FZhH06pBG_zC7jFk7ANRFz5MYCt7mT-xX1FeZUA28_TYBD08oKB3Pn5n_K7aLg=s170")

Tutorial adaptado del tutorial original de Naima: https://naima.readthedocs.io/en/latest/tutorial.html

Naima use the packages:

In [11]:
A1

NumPy es funcdamental para hacer computación científica. Es muy útil porque tiene arrays N dimencionales.

In [12]:
A2

SciPy es un conjunto de paquete para hacer matemáticas y ciencia en general.

In [15]:
A3

Matplotlib es un paquete que produce gráficos en 2D en Python.

# Corner.py

Es un paquete para visualizar muestras multidimensionales con una matriz de scatter plot para ver las covarianzas. Corner fue originalmente hecho para mostrar resultados de simulaciones de Moarkov Chain Monte Carlo.

# h5py

Este paquete es una interface de Python al formato de data binario HDF5. Permite manipular de manera muy facil la data si se tiene grandes cantidades de data numérica.

# emcee

Es un paquete licensiado por el MIT para producir y manipular ensembles Goodman & Weare’s Affine Invariant Markov chain Monte Carlo (MCMC).

El paquete actual (3.0.2) no permite utilizar la función de  ``naima.run_sampler`` por lo cual se recomienda bajar la 2.1.0

# Tutorial Extendido: Modelar un espectro SED con Naima

### 1° Primero se debe leer el espectro en un formato adecuado

La tabla con los datos debe contener al menos 3 columnas con las unidades apropiadas:
- ``Energía``: Energía de fotones observada [``Energía ``: $TeV$]
- ``Flujo``: Flujo observado [``flujo`` : $erg\,cm^{−2}\,s^{−1}$ o
  ``flujo diferencial`` :  $1/(s\,cm^2\,eV)$]
- ``Error del flujo``: 68% CL gaussian uncertainty of the flux [``flujo`` : $erg\,cm^{−2}\,s^{−1}$ o
  ``flujo diferencial`` :  $1/(s\,cm^2\,eV)$]. O en su lugar se puede aceptar: ``flux_error_lo``
  and ``flux_error_hi``.

- ``flux_error_lo`` and ``flux_error_hi``: 68% CL gaussian lower and
  upper uncertainties of the flux.

In [16]:
import numpy as np
from astropy.io import ascii
import naima

C:\Users\Ariadna\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [17]:
## Read data

data = ascii.read("RXJ1713_HESS_2007.dat")

In [18]:
print(data)

energy energy_edge_lo energy_edge_hi       flux         flux_error    ul
 TeV        TeV            TeV       1 / (cm2 s TeV) 1 / (cm2 s TeV)    
------ -------------- -------------- --------------- --------------- ---
  0.33            0.3           0.37        2.29e-10         3.2e-11   0
   0.4           0.37           0.44        1.25e-10         1.6e-11   0
  0.49           0.44           0.54        9.46e-11           9e-12   0
  0.59           0.54           0.65        6.06e-11         5.2e-12   0
  0.71           0.65           0.79        4.37e-11         3.1e-12   0
  0.86           0.79           0.95        2.15e-11         1.8e-12   0
  1.04           0.95           1.15        1.82e-11         1.1e-12   0
  1.26           1.15           1.39        1.17e-11           7e-13   0
  1.53           1.39           1.69        8.87e-12           5e-13   0
  1.85           1.69           2.04        5.63e-12         3.3e-13   0
   ...            ...            ...             ..

Muchos tipos de tablas de datos pueden ser utilizadas por las funciones ``get_sampler`` y ``run_sampler``. Estas tablas deben cumplir los requerimientos de arriba, pero no necesariamente tienen que estar en las mismas unidades porque Naima puede cambiarlas apropiadamente. 

Naima hace uso de Astropy para la conversión de unidades : ``astropy.units``. Muchos de los argumentos de las funciones y classes de Naima requieren cantidades en formatos definidos. Estas cantidades se pueden definir como:

In [19]:
from astropy import units as u
import numpy as np

# Define scalar quantity
q1 = 3. * u.kpc

# Define array quantity using a list
q2 = [1., 2., 3.] * u.arcsec

# Define array quantity using a Numpy array
q3 = np.array([1., 2., 3.]) * u.cm ** 2 / u.g

### 2° Construyendo el modelo

La función del modelo es la función que debe ser utilizada para modelar el espectro observable. Esta función debe aceptar dos parametros:
- Un array parametros libres del modelo
- La tabla de datos

Naima incluye muchos modelos en el modulo ``naima.models`` que facilita modelar espectros comunes como:
- PowerLaw
- ExponentialCutoffPowerLaw
- BrokenPowerLaw
- LogParabola 

Tambien contiene muchos modelos de radiación como:

- InverseCompton
- Synchroton
- Bremsstrahlung
- PionDecay

Una vez que se inicializan los parametros necesarios, todos los modelos pueden ser llamados con un array de energía para obtener el flujo del modelo correspondientes a los valores de energía en el array. Si se le entrega la tabla de datos como argumento, los valores de energía de la columna de energía serán usados.

Construir la función del modelo a partir de los modelos de Naima es facil. En el ejemplo, hay tres parametros del modelo en el array ``pars`` son la amplitud, el indice espectral (espectral index) y la energía de corte (cutoff energy). Primero se añadern las unidades necesarias para los modelos y luego se procesan y retornan el modelo del flujo para las energías en la tabla de datos.

In [20]:
## Model definition
from naima.models import ExponentialCutoffPowerLaw, InverseCompton

## Model definition

def ElectronIC(pars, data):

    # Match parameters to ECPL properties, and give them the appropriate units
    amplitude = pars[0] / u.eV
    alpha = pars[1]
    e_cutoff = (10 ** pars[2]) * u.TeV

    # Initialize instances of the particle distribution and radiative model
    ECPL = ExponentialCutoffPowerLaw(amplitude, 10.0 * u.TeV, alpha, e_cutoff)
    # Compute IC on CMB and on a FIR component with values from GALPROP for the
    # position of RXJ1713
    IC = InverseCompton(
        ECPL,
        seed_photon_fields=[
            "CMB",
            ["FIR", 26.5 * u.K, 0.415 * u.eV / u.cm ** 3],
        ],
        Eemin=100 * u.GeV,
    )

    # compute flux at the energies given in data['energy'], and convert to units
    # of flux data
    model = IC.flux(data, distance=1.0 * u.kpc).to(data["flux"].unit)

    # Save this realization of the particle distribution function
    elec_energy = np.logspace(11, 15, 100) * u.eV
    nelec = ECPL(elec_energy)

    # Compute and save total energy in electrons above 1 TeV
    We = IC.compute_We(Eemin=1 * u.TeV)

    # The first array returned will be compared to the observed spectrum for
    # fitting. All subsequent objects will be stores in the sampler metadata
    # blobs.
    return model, (elec_energy, nelec), We

En adición a esto, se debe contruir una función ``prior``. Esta función produce el código que corresponde a cualquier conocimiento previo sobre los parámetros, como mediciones previas o rangos físicamente aceptables. Tres simples funciones ``prior`` estan incluidas en Naima: ``normal_prior``,  ``uniform_prior`` y ``loguniform_prior``.

In [21]:
	## Prior definition


def lnprior(pars):
    """
    Return probability of parameter values according to prior knowledge.
    Parameter limits should be done here through uniform prior ditributions
    """

    logprob = naima.uniform_prior(pars[0], 0.0, np.inf) + naima.uniform_prior(
        pars[1], -1, 5
    )

    return logprob

### 3° Eligiendo un punto de inicio para el Sampling

Antes de iniciar el Marcov Chain Monte Carlo, debemos darle los parametros iniciales estimados con los siguientes nombres en un formato de array y una lista:

In [22]:
if __name__ == "__main__":

    ## Set initial parameters and labels
    p0 = np.array((1e30, 3.0, np.log10(30)))
    labels = ["norm", "index", "log10(cutoff)"]

Para los modelos con más de un canal de emisión como Synchroton y Inverse Compron es más dificil dar los parametros sin compararlo visualmente con el espectro. Para estos modelos que los parametros iniciales (en forma de vector) estan lejos de los estimados por el maximum likelihood estimation puede significar que la minimización o el proceso de sampling el algoritmo se quedo atorado en un máximo local.

En orden de hacer facilmente una adecuada estimación, Naima provee una herramienta para interactivamente ver el output del modelo y compararlo  con la data observada mientras se cambian los parametros(en forma de vector).  Esta herramienta se activa al poner ``interactive=True`` in the options of ``get_sampler`` or ``run_sampler``. Esto abrira una ventana de matplotlib interactiva en la cual los parametros pueden ser cambiados y eso modifica el modelo resultante al espectro input. En cada cambio la probabilidad (log) del modelo  dado el espectro input es recalculada. En adición, el Nelder-Mead fit puede ser abierto desde un boton en la ventana para encontrar los parametros estimados (en forma de vector) de maximum likelihood. Una vez que el usuario considera que el modelo es una buena aproximación del espectro input, debe cerrar la ventana y se usara el parametro que haya elegido para el proceso de ``sampling``(ver el siguiente caso).

Un modo alternativo de acceder a la herramienta alternativa es acceder a ella directamente a través de la class ``naima.InteractiveModelFitter``  a traves de un interpter de Python interactivo. Los parametros(en forma de vector) se mostraran en una ventana interactiva a la que se puede acceder a traves del atributo ``imf.pars`` y de ahi ser copiados al nuevo p0 para luego utilizarla en el sampling. Todo este proceso se realiza del siguiente modo:

    imf = InteractiveModelFitter(model, p0, data=data, labels=labels)
    p0 = imf.pars
    
Notese que se puede omitir el argumento de data al especificar un rango en el parametro ``e_range`` para hacer el ajuste independientemente de la data:

    imf = InteractiveModelFitter(model, p0, e_range=[1*u.GeV, 100*u.TeV])

### 4° Sampling la función del modelo

Debemos usar las funciones ``get_sampler`` y ``run_sampler`` en la tabla del input.

Todos los objetos obtenidos anteriormente deben ser dados a la función ``run_sampler`` que corre un sampler del MCMC, la principal función para modelar ('fitting') en Naima:

El parametro de los ``nwalkers`` ('walkers': parametro del paquete de emcee. Son integrantes del ensamble) especifican cuantos parametros deben ser usados en el sampling. El parametro ``nburn`` especifica cuantos pasos deben avanzar y los parametros ``burn-in`` y ``nrun`` especifican cuantos pasos deberian avanzar despues de que se acaban los pasos y estos parametros se guardan en el objeto ``sampler``.

In [23]:
if __name__ == "__main__": 
    ## Run sampler
    sampler, pos = naima.run_sampler(
        data_table=data,
        p0=p0,
        labels=labels,
        model=ElectronIC,
        prior=lnprior,
        nwalkers=32,
        nburn=100,
        nrun=20,
        threads=4,
        prefit=True,
    )

    ## Save run results to HDF5 file (can be read later with naima.read_run)
    naima.save_run("RXJ1713_IC_run.hdf5", sampler)

INFO: Finding Maximum Likelihood parameters through Nelder-Mead fitting... [naima.core]
INFO:    Initial parameters: [1.87859806e+32 3.00000000e+00 1.47712125e+00] [naima.core]
INFO:    Initial lnprob(p0): -175.747 [naima.core]
INFO:    New ML parameters : [1.38844453e+32 2.57175369e+00 1.68211459e+00] [naima.core]
INFO:    Maximum lnprob(p0): -18.019 [naima.core]
Burning in the 32 walkers with 100 steps...


KeyboardInterrupt: 

### 5° Inspecting and analysing results of the run

Los resultados son guardados en un objeto llamado ``sampler`` que puede ser analizado a través de las funciones para plotear de Naima: ``plot_chain``, ``plot_fit``, y ``plot_data``. En adición, dos funciones convenientes pueden ser usadas para generar una coleción de plots que ilustran los resultados y la estabilidad del procedimeinto. Estos son ``save_diagnostic_plots`` y ``save_results_table``.

La tabla guardada incluira información en la metadata acerca del proceso como el número de walkers ``n_walkers`` y pasos ``n_run`` usados en el sampling, los parametros iniciales (en forma de vector), el parametro vector con la maximum likelihood ``ML_pars`` y el maxixmo valor deel negativo negative log-likelihood ``MaxLogLikelihood``. La tabla en si misma muestra las media y upper & lower incertidumbres (50th, 84th y 16th percentiles de la distribución) para los parametros del espectro input.

La tabla es guardada por default con un formato ECSV la cual puede ser facilmente accesible con el modulo ``astropy.io.ascii``.

In [ ]:
if __name__ == "__main__":
    ## Diagnostic plots with labels for the metadata blobs
    naima.save_diagnostic_plots(
        "RXJ1713_IC",
        sampler,
        sed=True,
        last_step=False,
        blob_labels=[
            "Spectrum",
            "Electron energy distribution",
            "$W_e (E_e>1\, \mathrm{TeV})$",
        ],
    )
    naima.save_results_table("RXJ1713_IC", sampler)